<a href="https://colab.research.google.com/github/SRI-CSL/signal-public/blob/signal-demonstration/colabs/signal_api_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SIGNAL**ing Interest Data

**Description:** Installation, basic APIs, and Dashboards, see [HERE](https://github.com/SRI-CSL/SIGNAL/blob/main/reports/milestone-7/signal-demo.org)

**Copyright 2022 SRI International.**

This project is under the GPL3 License. See the [LICENSE](https://www.gnu.org/licenses/gpl-3.0.en.html) file for the full license text.

## &#128640; Getting Started

Install the `SIGNAL API` client

In [ ]:
!curl https://signal.cta.sri.com/client > client.tgz
!tar xzf client.tgz
!pip install -r signal_api_client/requirements.txt
!pip install -e signal_api_client
!pip install ipympl
%cd /content/signal_api_client   

## &#9776; Dependencies

In [22]:
import os
import sys

In [23]:
import time
import warnings

import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import List, Any, Dict, Tuple
from datetime import date, datetime

In [64]:
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

In [47]:
from sqlalchemy.orm import Session

In [25]:
from signal_api import signal

In [26]:
import signal_api.util as utils

## &#9997; Configuration

In [27]:
warnings.filterwarnings("ignore")

In [28]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [29]:
np.random.seed(0)

In [30]:
pd.options.plotting.backend = "plotly"

In [31]:
from IPython import get_ipython
utils.common.set_default_vars(os.environ, ipython_val=get_ipython())

In [32]:
utils.console.stdout.print(utils.common.is_run_in_colab())

True

## &#128272; Login

In [48]:
# from datetime import datetime
# from sqlalchemy.orm import Session
# from signal_api import signal

signal.login(host='signal.cta.sri.com')

True

## Functions

In [34]:
def bar_plot(value_series: pd.Series, log_y: bool=True, labels: Dict={}, show_legend: bool=False) -> None:
  fig = value_series.plot.bar(log_y=log_y, labels=labels)
  fig.update_traces(showlegend=show_legend)  
  fig.show()

In [35]:
def df_groupby(input_df: pd.DataFrame, groupby_column: str, sort_values: bool=True) -> pd.Series:
    if sort_values:
        tmp_data = input_df.groupby(input_df[groupby_column]).size().sort_values()
    else:
        tmp_data = input_df.groupby(input_df[groupby_column]).size()
    
    return tmp_data

In [36]:
def get_record_count(table_name: str) -> int:
    query = f"SELECT COUNT(*) FROM {table_name};"
    df_result = signal.query_dataframe(query)
    result = df_result['count'].iloc[0]
    return result

In [37]:
def get_table_columns(table_name: str) -> List[str]:
    query = f"SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'{table_name}';"
    df_result = signal.query_dataframe(query)
    result_list = df_result.column_name.tolist()
    return result_list

In [38]:
def print_table_info(table_name: str) -> None:
    total_records = get_record_count(table_name=table_name)
    table_columns = get_table_columns(table_name=table_name)

    print(f"- In total, there are {total_records:,} records in the {table_name} table.\n"\
          f"-- Table {table_name} contains {len(table_columns)} columns, namely: {table_columns}.\n")

In [39]:
def get_db_records(query: str) -> pd.DataFrame:
    df_result = signal.query_dataframe(query)

    return df_result

In [58]:
def get_project(signal_api: signal, project_name: str='linux_kernel') -> signal.projects:
  session = Session(signal_api.database.engine)
  if project_name == 'linux_kernel':
    result = signal_api.projects(session)[0]
    return result
  else:
    raise Exception(f"Project {project_name} is not currently supported!")

In [90]:
def plot_project_stats(
  project: signal.projects, 
  start_date: datetime, 
  end_date: datetime, 
  features: List=['authors', 'commits', 'small_changes', 'medium_changes', 'big_changes', 
                  'important_changes', 'tedious_changes', 'dense_changes', 'new_funcs', 
                  'cleanup', 'bug_fixes', 'triggers']) -> pd.DataFrame:

  history = project.git_history(start_date, end_date)
  history_data = history.histogram()
  result_df = pd.DataFrame(history_data)

  fig = make_subplots(
    rows=4, 
    cols=3, 
    subplot_titles=('Authors', 'Commits', 'Small Changes',  'Medium Changes', 'Big Changes', 'Important Changes', 'Tedious Changes', 'Dense Changes', 'New Functionalities', 'Cleanup', 'Bug Fixes', 'Triggers'))
  
  r = 0
  c = 0
  for i, feature in enumerate(features):
    if i % 3 == 0:
      r += 1
      c = 1
    else:
      c += 1
    
    fig.add_trace(go.Bar(x=result_df.time_period, y=result_df[feature]), row=r, col=c)

  fig.update_layout(title_text="SIGNAL Data Statistics", height=950, showlegend=False)
  fig.show()

  return result_df, result_df.describe()

## &#129504; Main Plots

In [60]:
linux_kernel = get_project(signal_api=signal, project_name='linux_kernel')

In [93]:
stats_df, metrics = plot_project_stats(project=linux_kernel, start_date=datetime(2020, 1, 1), end_date=datetime(2020, 12, 31))

In [94]:
metrics

,authors,commits,small_changes,medium_changes,big_changes,important_changes,tedious_changes,dense_changes,new_funcs,cleanup,bug_fixes,triggers
count,53.000000,53.000000,53.0,53.00000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.0
mean,433.377358,1577.433962,0.0,1153.09434,424.339623,1548.433962,728.811321,723.433962,310.377358,84.056604,1182.811321,0.0
std,112.929731,525.464758,0.0,371.81469,160.623059,518.046534,246.033773,248.675763,110.423873,31.123461,393.195296,0.0
min,121.000000,308.000000,0.0,248.00000,60.000000,304.000000,198.000000,144.000000,87.000000,10.000000,211.000000,0.0
25%,376.000000,1330.000000,0.0,995.00000,336.000000,1300.000000,595.000000,610.000000,233.000000,66.000000,1020.000000,0.0
50%,464.000000,1622.000000,0.0,1188.00000,414.000000,1601.000000,727.000000,746.000000,328.000000,86.000000,1206.000000,0.0
75%,504.000000,2004.000000,0.0,1432.00000,547.000000,1962.000000,929.000000,930.000000,391.000000,107.000000,1486.000000,0.0
max,616.000000,2389.000000,0.0,1737.00000,697.000000,2340.000000,1161.000000,1150.000000,488.000000,146.000000,1796.000000,0.0


## &#128722; Data

### &#9759; Tables

In [40]:
TABLES_QUERY = "SELECT * FROM information_schema.tables WHERE table_type='BASE TABLE';"

In [41]:
df_tables = signal.query_dataframe(TABLES_QUERY)

In [42]:
table_names = df_tables.table_name.unique()

In [43]:
print(f"There are {len(table_names)} tables currently present in the SIGNAL database.")

There are 87 tables currently present in the SIGNAL database.


In [44]:
df_tables.head()

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,signal,public,scraped_projects,BASE TABLE,None,None,None,None,None,YES,NO,None
1,signal,public,scraped_patch_series,BASE TABLE,None,None,None,None,None,YES,NO,None
2,signal,public,diff,BASE TABLE,None,None,None,None,None,YES,NO,None
3,signal,public,thread,BASE TABLE,None,None,None,None,None,YES,NO,None
4,signal,public,git_files,BASE TABLE,None,None,None,None,None,YES,NO,None


In [45]:
table_names

array(['scraped_projects', 'scraped_patch_series', 'diff', 'thread',
       'git_files', 'email', 'git_commit_edges', 'pg_statistic',
       'pg_type', 'git_file_changes', 'scraped_patches',
       'alembic_version', 'api_key', 'pg_foreign_table', 'pg_authid',
       'signal_history', 'pg_statistic_ext_data', 'git_repos',
       'user_identities', 'person', 'scraped_patch_submitters',
       'mailing_list', 'git_sigs', 'pg_user_mapping', 'pg_subscription',
       'pg_attribute', 'pg_proc', 'pg_class', 'pg_attrdef',
       'pg_constraint', 'pg_inherits', 'pg_index', 'pg_operator',
       'pg_opfamily', 'pg_opclass', 'pg_am', 'pg_amop', 'pg_amproc',
       'pg_language', 'pg_largeobject_metadata', 'pg_aggregate',
       'pg_statistic_ext', 'pg_rewrite', 'pg_trigger', 'pg_event_trigger',
       'pg_description', 'pg_cast', 'pg_enum', 'pg_namespace',
       'pg_conversion', 'pg_depend', 'pg_database', 'pg_db_role_setting',
       'pg_tablespace', 'pg_auth_members', 'pg_shdepend',
       'p

In [46]:
print_table_info(table_name='email')
print_table_info(table_name='scraped_projects')
print_table_info(table_name='scraped_patch_series')
print_table_info(table_name='diff')
print_table_info(table_name='thread')
print_table_info(table_name='git_files')
print_table_info(table_name='git_commit_edges')
print_table_info(table_name='git_file_changes')
print_table_info(table_name='scraped_patches')
print_table_info(table_name='git_repos')
print_table_info(table_name='user_identities')
print_table_info(table_name='person')
print_table_info(table_name='scraped_patch_submitters')
print_table_info(table_name='mailing_list')
print_table_info(table_name='git_sigs')
print_table_info(table_name='patchwork_person_id')
print_table_info(table_name='git_commits')
print_table_info(table_name='git_xrefs')

- In total, there are 828,219 records in the email table.
-- Table email contains 15 columns, namely: ['id', 'mailing_list_id', 'author_id', 'timestamp_sent', 'timestamp_recv', 'reply_to_url', 'thread_id', 'persuasion', 'reply_to_message_id', 'subject', 'body', 'url', 'clean_body', 'email_id', 'message_id'].

- In total, there are 93 records in the scraped_projects table.
-- Table scraped_projects contains 2 columns, namely: ['id', 'name'].

- In total, there are 280,722 records in the scraped_patch_series table.
-- Table scraped_patch_series contains 3 columns, namely: ['id', 'patchwork_id', 'name'].

- In total, there are 667,972 records in the diff table.
-- Table diff contains 4 columns, namely: ['id', 'email_id', 'file', 'contents'].

- In total, there are 117,508 records in the thread table.
-- Table thread contains 20 columns, namely: ['id', 'deg_max', 'deg_max_2', 'deg_max_3', 'deg_max_4', 'deg_max_5', 'patch', 'emails', 'users', 'start', 'days', 'depth', 'star_nodes', 'h_index

## &#129504; Plots

### &#9759; person Table

In [28]:
# person table
df_person = get_db_records("SELECT * FROM person;")

In [29]:
df_person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21345 entries, 0 to 21344
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21345 non-null  int64  
 1   name           21345 non-null  object 
 2   email_address  21345 non-null  object 
 3   is_bot         21345 non-null  int64  
 4   cluster_id     17689 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 833.9+ KB


In [30]:
df_person.head()

,id,name,email_address,is_bot,cluster_id
0,24,kernel test robot,lkp@intel.com,1,NaN
1,46,pr-tracker-bot@kernel.org,pr-tracker-bot@kernel.org,1,NaN
2,113,kernel test robot,lkp@ff58d72860ac,1,NaN
3,141,kernel test robot,lkp@shao2-debian,1,NaN
4,144,kernel test robot,rong.a.chen@shao2-debian,1,NaN


In [31]:
tmp_person_is_bot = df_groupby(input_df=df_person, groupby_column='is_bot')
tmp_person_is_bot

is_bot
1     3656
0    17689
dtype: int64

In [32]:
tmp_person_is_bot

is_bot
1     3656
0    17689
dtype: int64

In [33]:
bar_plot(value_series=tmp_person_is_bot, log_y=False, labels=dict(is_bot="User Type", value="number of records"))

### &#9759; scraped_patches Table

In [34]:
df_scraped_patches = get_db_records("SELECT * FROM scraped_patches;")

In [35]:
df_scraped_patches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 938137 entries, 0 to 938136
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             938137 non-null  int64  
 1   project_id     938137 non-null  int64  
 2   patch_id       938137 non-null  int64  
 3   mail_id        938137 non-null  object 
 4   name           938137 non-null  object 
 5   series_id      935162 non-null  float64
 6   ack_count      938137 non-null  int64  
 7   review_count   938137 non-null  int64  
 8   tested_count   938137 non-null  int64  
 9   success_count  938137 non-null  int64  
 10  warning_count  938137 non-null  int64  
 11  fail_count     938137 non-null  int64  
 12  date           938137 non-null  object 
 13  submitter_id   938137 non-null  int64  
 14  delegate       125878 non-null  object 
 15  state          938137 non-null  object 
 16  commit_hash    48390 non-null   object 
dtypes: float64(1), int64(10), obj

In [36]:
df_scraped_patches.head()

,id,project_id,patch_id,mail_id,name,series_id,ack_count,review_count,tested_count,success_count,warning_count,fail_count,date,submitter_id,delegate,state,commit_hash
0,3488,59,13000777,20221006220840.275-4-jonathan.derrick@linux.dev,[2/2] md/bitmap: Add chunk-count-based bitmap ...,1446.0,0,0,0,0,0,0,2022-10-06T00:00:00,522,None,Superseded,None
1,3489,56,13034662,20221107155825.1644604-12-pierre.gondois@arm.com,"[v2,11/23] arm64: dts: Update cache properties...",1203.0,0,1,0,0,0,0,2022-11-07T00:00:00,411,geert,New,None
2,3490,56,13034661,20221107155825.1644604-11-pierre.gondois@arm.com,"[v2,10/23] arm64: dts: Update cache properties...",1203.0,0,0,0,0,0,0,2022-11-07T00:00:00,411,geert,New,None
3,3491,57,13004984,20221012114429.2341215-6-danishanwar@ti.com,"[v6,5/5] remoteproc: pru: Configure firmware b...",1447.0,0,0,0,0,0,0,2022-10-12T00:00:00,396,None,Superseded,None
4,3492,57,13004985,20221012114429.2341215-5-danishanwar@ti.com,"[v6,4/5] remoteproc: pru: Add pru_rproc_set_ct...",1447.0,0,0,0,0,0,0,2022-10-12T00:00:00,396,None,Superseded,None


In [37]:
tmp_sp_project_id = df_groupby(input_df=df_scraped_patches, groupby_column='project_id')
tmp_sp_project_id

project_id
63         7
16         8
77        19
29        41
17        56
       ...  
52     39650
87     49635
31     70477
76     95940
58    115437
Length: 93, dtype: int64

In [38]:
bar_plot(value_series=tmp_sp_project_id, labels=dict(project_id="Project ID", value="number of records"))

In [39]:
tmp_sp_state = df_groupby(input_df=df_scraped_patches, groupby_column='state')
tmp_sp_state

state
Needs ACK                48
In Next                 124
Under Review            744
Queued                  941
Rejected               2136
Deferred               4252
RFC                    6300
Awaiting Upstream      7308
Handled Elsewhere      7809
Mainlined             14739
Changes Requested     26867
Not Applicable        49217
Accepted              91480
Superseded           135830
New                  590342
dtype: int64

In [40]:
bar_plot(value_series=tmp_sp_state, labels=dict(state="State", value="number of records"))

In [70]:
tmp_sp_project_state = df_scraped_patches.groupby([df_scraped_patches.project_id, df_scraped_patches.state]).count()
tmp_sp_project_state

id  patch_id  mail_id  name  series_id  \
project_id state                                                        
1          New                253       253      253   253        253   
2          New                186       186      186   186        186   
3          Accepted           348       348      348   348        348   
           Changes Requested    4         4        4     4          4   
           New                 73        73       73    73         73   
...                           ...       ...      ...   ...        ...   
93         Handled Elsewhere   89        89       89    89         89   
           In Next              1         1        1     1          1   
           New                 23        23       23    23         23   
           Rejected             2         2        2     2          2   
           Superseded         296       296      296   296        296   

                              ack_count  review_count  tested_count  \
project_id state                                                      
1          New                      253           253           253   
2          New                      186           186           186   
3          Accepted                 348           348           348   
           Changes Requested          4             4             4   
           New                       73            73            73   
...                                 ...           ...           ...   
93         Handled Elsewhere         89            89            89   
           In Next                    1             1             1   
           New                       23            23            23   
           Rejected                   2             2             2   
           Superseded               296           296           296   

                              success_count  warning_count  fail_count  date  \
project_id state                                                               
1          New                          253            253         253   253   
2          New                          186            186         186   186   
3          Accepted                     348            348         348   348   
           Changes Requested              4              4           4     4   
           New                           73             73          73    73   
...                                     ...            ...         ...   ...   
93         Handled Elsewhere             89             89          89    89   
           In Next                        1              1           1     1   
           New                           23             23          23    23   
           Rejected                       2              2           2     2   
           Superseded                   296            296         296   296   

                              submitter_id  delegate  commit_hash  
project_id state                                                   
1          New                         253         0           95  
2          New                         186         0           86  
3          Accepted                    348         0          163  
           Changes Requested             4         0            0  
           New                          73         0            0  
...                                    ...       ...          ...  
93         Handled Elsewhere            89         0            0  
           In Next                       1         0            0  
           New                          23         0            0  
           Rejected                      2         0            0  
           Superseded                  296         0            0  

[602 rows x 15 columns]

In [71]:
bar_plot(value_series=tmp_sp_project_state.id.unstack(), log_y=True, labels=dict(project_id="Project ID", value="number of records", state="Patch States"), show_legend=True)

### &#9759; threads Table

In [43]:
df_thread = get_db_records("SELECT * FROM thread;")

In [44]:
df_thread.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117508 entries, 0 to 117507
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               117508 non-null  int64  
 1   name             117508 non-null  object 
 2   root_message_id  117508 non-null  object 
 3   url              117508 non-null  object 
 4   subject          117508 non-null  object 
 5   patch            117508 non-null  int64  
 6   emails           117508 non-null  int64  
 7   users            117508 non-null  int64  
 8   start            117508 non-null  int64  
 9   days             117508 non-null  int64  
 10  depth            117508 non-null  int64  
 11  star_nodes       117508 non-null  int64  
 12  h_index          117508 non-null  int64  
 13  deg_gini         117508 non-null  float64
 14  deg_max          117508 non-null  float64
 15  deg_max_2        85094 non-null   float64
 16  deg_max_3        58892 non-null   floa

In [45]:
df_thread.head()

,id,name,root_message_id,url,subject,patch,emails,users,start,days,depth,star_nodes,h_index,deg_gini,deg_max,deg_max_2,deg_max_3,deg_max_4,deg_max_5,category
0,1,20200817155032,20200817143833.737102804@linuxfoundation.org,https://lkml.iu.edu/hypermail/linux/kernel/200...,[PATCH 5.8 001/464] tracepoint: Mark __tracepo...,1,475,9,1597677533,1,3,1,1,1.0,467.0,1.0,1.0,1.0,1.0,comet
1,2,20200817151017,20200817143819.579311991@linuxfoundation.org,https://lkml.iu.edu/hypermail/linux/kernel/200...,[PATCH 5.7 001/393] tracepoint: Mark __tracepo...,1,398,6,1597679578,1,2,1,1,1.0,396.0,1.0,0.0,0.0,0.0,comet
2,3,20200929081604,20200929110010.467764689@linuxfoundation.org,https://lkml.iu.edu/hypermail/linux/kernel/200...,[PATCH 5.4 006/388] drm/amd/display: Do not do...,1,389,3,1601379429,0,1,1,1,1.0,388.0,0.0,0.0,0.0,0.0,comb
3,4,20200917220115,20200918020110.2063155-1-sashal@kernel.org,https://lkml.iu.edu/hypermail/linux/kernel/200...,[PATCH AUTOSEL 5.4 001/330] drm/v3d: don't lea...,1,340,10,1600394475,7,4,2,2,1.0,329.0,2.0,1.0,1.0,1.0,tree
4,5,20200817143209,20200817143755.807583758@linuxfoundation.org,https://lkml.iu.edu/hypermail/linux/kernel/200...,[PATCH 5.4 014/270] arm64: dts: qcom: msm8916:...,1,275,6,1597682723,1,2,1,1,1.0,273.0,1.0,0.0,0.0,0.0,comet


In [46]:
tmp_thread_category = df_groupby(input_df=df_thread, groupby_column='category')
tmp_thread_category

category
waterfall      326
stringy      12549
tree         12657
comet        20977
atom         32414
comb         38585
dtype: int64

In [47]:
bar_plot(value_series=tmp_thread_category, log_y=False, labels=dict(category="Thread Category", value="number of records"))

## &#9759; Stats Plots

In [48]:
START_DATE = datetime(2020, 8, 1)
END_DATE = datetime(2020, 8, 10)
# df_email = signal.query_dataframe(f"SELECT * FROM email WHERE timestamp_sent > {START_DATE.timestamp()} and timestamp_sent < {END_DATE.timestamp()};")

* **EMAIL** $E$: ['id', 'mailing_list_id', 'author_id', 'timestamp_sent', 'timestamp_recv', 'reply_to_url', 'thread_id', 'persuasion', 'reply_to_message_id', 'subject', 'body', 'url', 'clean_body', 'email_id', 'message_id']

* **SCRAPED PROJECTS** $S$: ['id', 'name']

* **SCRAPED PATCHES** $P$: ['id', 'project_id', 'patch_id', 'review_count', 'tested_count', 'success_count', 'warning_count', 'fail_count', 'submitter_id', 'series_id', 'ack_count', 'mail_id', 'name', 'commit_hash', 'delegate', 'state', 'date']

* **PERSON** $A$: ['id', 'is_bot', 'cluster_id', 'name', 'email_address']

*SQL Query*

```sql
SELECT E.mailing_list_id, E.author_id, A.name, A.email_address, A.is_bot, E.timestamp_sent, E.timestamp_recv, E.reply_to_url, E.thread_id, E.persuasion, E.reply_to_message_id, E.email_id, E.message_id, P.project_id, P.patch_id, P.review_count, P.tested_count, P.success_count, P.warning_count, P.fail_count, P.submitter_id, P.series_id, P.ack_count, P.mail_id, P.name AS patch_name, P.commit_hash, P.delegate, P.state, P.date, S.name
FROM email E, scraped_projects S, scraped_patches P, person A
WHERE 
  E.timestamp_sent >= START_DATE 
  AND E.timestamp_sent <= END_DATE
  AND E.message_id = P.mail_id
  AND E.author_id = A.id
  AND P.project_id = S.id
```



These are the default columns of `report/metrics`

* metrics returned via `get_project(name: str="linux kernel", columns: List, plot_summary:bool=True)`
- no. commits
- no. patches that were accepted and rejected
- no. of mailing lists
- no. of contributors
- no. of maintainers
- from when to when data range
- no. of bots
- no. of persuasive emails
- project name

API will return two values: 
1. the name: a tuple containing the name of the project and its ID
2. the report: 3 attributes: [metrics (will match the columns that you are specifying), start_date, end_date]
  - will only provide a grid of only numeric columns


Once this is done: `contribution_df` is next and additional data for pelt plot


In [49]:
stats_query = f"SELECT E.mailing_list_id, E.author_id, A.name, A.email_address, "\
              f"A.is_bot, E.timestamp_sent, E.timestamp_recv, E.reply_to_url, "\
              f"E.thread_id, E.persuasion, E.reply_to_message_id, E.email_id, E.message_id, "\
              f"P.project_id, P.patch_id, P.review_count, P.tested_count, P.success_count, "\
              f"P.warning_count, P.fail_count, P.submitter_id, P.series_id, P.ack_count, "\
              f"P.mail_id, P.name AS patch_name, P.commit_hash, P.delegate, P.state, P.date, S.name AS project_name "\
              f"FROM email E, scraped_projects S, scraped_patches P, person A "\
              f"WHERE "\
              f"E.timestamp_sent >= {START_DATE.timestamp()} "\
              f"AND E.timestamp_sent <= {END_DATE.timestamp()} "\
              f"AND E.message_id = P.mail_id "\
              f"AND E.author_id = A.id "\
              f"AND P.project_id = S.id;"
stats_query                        

'SELECT E.mailing_list_id, E.author_id, A.name, A.email_address, A.is_bot, E.timestamp_sent, E.timestamp_recv, E.reply_to_url, E.thread_id, E.persuasion, E.reply_to_message_id, E.email_id, E.message_id, P.project_id, P.patch_id, P.review_count, P.tested_count, P.success_count, P.warning_count, P.fail_count, P.submitter_id, P.series_id, P.ack_count, P.mail_id, P.name AS patch_name, P.commit_hash, P.delegate, P.state, P.date, S.name AS project_name FROM email E, scraped_projects S, scraped_patches P, person A WHERE E.timestamp_sent >= 1596240000.0 AND E.timestamp_sent <= 1597017600.0 AND E.message_id = P.mail_id AND E.author_id = A.id AND P.project_id = S.id;'

In [50]:
df_stats = get_db_records(query=stats_query)

In [51]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2486 entries, 0 to 2485
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   mailing_list_id      2486 non-null   int64 
 1   author_id            2486 non-null   int64 
 2   name                 2486 non-null   object
 3   email_address        2486 non-null   object
 4   is_bot               2486 non-null   int64 
 5   timestamp_sent       2486 non-null   int64 
 6   timestamp_recv       2486 non-null   int64 
 7   reply_to_url         1825 non-null   object
 8   thread_id            2486 non-null   object
 9   persuasion           2486 non-null   object
 10  reply_to_message_id  2486 non-null   object
 11  email_id             2486 non-null   object
 12  message_id           2486 non-null   object
 13  project_id           2486 non-null   int64 
 14  patch_id             2486 non-null   int64 
 15  review_count         2486 non-null   int64 
 16  tested

In [52]:
df_stats.head()

,mailing_list_id,author_id,name,email_address,is_bot,timestamp_sent,timestamp_recv,reply_to_url,thread_id,persuasion,...,submitter_id,series_id,ack_count,mail_id,patch_name,commit_hash,delegate,state,date,project_name
0,1,416,David Howells,dhowells@warthog.procyon.org.uk,0,1596809221,1596812821,None,159680892602.29015.6551860260436544999.stgit@w...,Unknown,...,652,179375,0,159680892602.29015.6551860260436544999.stgit@w...,[1/5] Add manpage for open_tree(2),None,None,New,2020-08-07T00:00:00,linux-fsdevel
1,1,69,Krzysztof Kozlowski,krzk@kernel.org,0,1596738988,1596742588,None,20200806181932.2253-1-krzk@kernel.org,Unknown,...,2630,78315,0,20200806182059.2431-17-krzk@kernel.org,"[v2,17/41] ARM: s5pv210: use private pm save/r...",423c62bfb623cb8b41afb0218bf68da56c4ab32e,None,Accepted,2020-08-06T00:00:00,linux-samsung-soc
2,1,69,Krzysztof Kozlowski,krzk@kernel.org,0,1596738988,1596742588,None,20200806181932.2253-1-krzk@kernel.org,Unknown,...,2630,269954,0,20200806182059.2431-17-krzk@kernel.org,"[v2,17/41] ARM: s5pv210: use private pm save/r...",None,None,New,2020-08-06T00:00:00,linux-arm-kernel
3,1,1713,Jonathan Adams,jwadams@google.com,0,1596835835,1596839435,https://lkml.iu.edu/hypermail/linux/kernel/200...,20200807212916.2883031-1-jwadams@google.com,Unknown,...,8648,179310,0,20200807212916.2883031-6-jwadams@google.com,"[RFC,5/7] core/metricfs: expose scheduler stat...",None,None,New,2020-08-07T00:00:00,linux-fsdevel
4,1,1713,Jonathan Adams,jwadams@google.com,0,1596835835,1596839435,https://lkml.iu.edu/hypermail/linux/kernel/200...,20200807212916.2883031-1-jwadams@google.com,Unknown,...,8648,210498,0,20200807212916.2883031-6-jwadams@google.com,"[RFC,5/7] core/metricfs: expose scheduler stat...",None,None,New,2020-08-07T00:00:00,kvm


In [53]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [54]:
stats_bot = df_groupby(input_df=df_stats, groupby_column='is_bot', sort_values=True)
stats_bot

is_bot
0    2486
dtype: int64

In [55]:
stats_state = df_groupby(input_df=df_stats, groupby_column='state', sort_values=True)
stats_state

state
Awaiting Upstream       3
Deferred                6
RFC                     9
Rejected               28
Changes Requested      43
Mainlined              47
Not Applicable        150
Accepted              153
Superseded            321
New                  1726
dtype: int64

In [56]:
stats_name = df_groupby(input_df=df_stats, groupby_column='project_name', sort_values=True)
stats_name

project_name
ath10k                1
ceph-devel            1
qemu-devel            1
linux-wpan            1
cifs-client           2
                   ... 
linux-fsdevel       125
linux-mediatek      127
dri-devel           156
linux-mm            180
linux-arm-kernel    408
Length: 61, dtype: int64

In [57]:
stats_persuasion = df_groupby(input_df=df_stats, groupby_column='persuasion', sort_values=True)
stats_persuasion

persuasion
Unknown    2486
dtype: int64

In [58]:
fig = make_subplots(rows=2, cols=2, subplot_titles=('Real Developer or Bots',  'Patch State', 'Projects',  'Persuasion Strategies'))

fig.add_trace(
    go.Bar(x=stats_bot.index, y=stats_bot),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=stats_state.index, y=stats_state),
    row=1, col=2
)

fig.add_trace(
    go.Bar(x=stats_name.index, y=stats_name),
    row=2, col=1
)

fig.add_trace(
    go.Bar(x=stats_persuasion.index, y=stats_persuasion),
    row=2, col=2
)

# Update xaxis properties
fig.update_xaxes(title_text="Real Dev/Bot", row=1, col=1)
fig.update_xaxes(title_text="State", row=1, col=2)
fig.update_xaxes(title_text="Project Name", row=2, col=1)
fig.update_xaxes(title_text="Persuasion Strategy", row=2, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="no. records", row=1, col=1)
fig.update_yaxes(title_text="no. records", row=1, col=2, type="log")
fig.update_yaxes(title_text="no. records", row=2, col=1)
fig.update_yaxes(title_text="no. records", row=2, col=2)

fig.update_layout(title_text="SIGNAL Data Overview", height=950, showlegend=False)
fig.show()

# TESTS

In [13]:
from datetime import datetime
from sqlalchemy.orm import Session
from signal_api import signal

signal.login(host='signal.cta.sri.com')
# session = Session(signal.database.engine)
# linux = signal.projects(session)[0]
# history = linux.git_history(datetime(2020, 8, 1), datetime(2020, 9, 1))
# history.histogram()

username?: ··········
password?: ··········


True

In [14]:
session = Session(signal.database.engine)

In [15]:
linux = signal.projects(session)[0]

In [16]:
history = linux.git_history(datetime(2020, 8, 1), datetime(2020, 9, 1))
history.histogram()

[CommitHistogramPeriod(time_period='2020-30', authors=73, commits=150, small_changes=0, medium_changes=123, big_changes=27, important_changes=148, tedious_changes=51, dense_changes=65, new_funcs=23, cleanup=9, bug_fixes=118, triggers=0),
 CommitHistogramPeriod(time_period='2020-31', authors=357, commits=1065, small_changes=0, medium_changes=785, big_changes=280, important_changes=1043, tedious_changes=495, dense_changes=484, new_funcs=199, cleanup=40, bug_fixes=826, triggers=0),
 CommitHistogramPeriod(time_period='2020-32', authors=265, commits=750, small_changes=0, medium_changes=587, big_changes=163, important_changes=732, tedious_changes=368, dense_changes=314, new_funcs=128, cleanup=47, bug_fixes=575, triggers=0),
 CommitHistogramPeriod(time_period='2020-33', authors=453, commits=1548, small_changes=0, medium_changes=1142, big_changes=406, important_changes=1526, tedious_changes=665, dense_changes=727, new_funcs=333, cleanup=77, bug_fixes=1138, triggers=0),
 CommitHistogramPeriod(t

In [20]:
len(history.histogram())

6

In [116]:

import plotly.express as px
 
df = px.data.iris()
 
fig = px.bar(df, x="sepal_width", y="sepal_length",
             color="species", barmode="group",
             facet_row="species", facet_col="species_id")
 
fig.show()

In [67]:
categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()

In [68]:
fig = go.Figure(data=go.Scatterpolar(
  r=[1, 5, 2, 2, 3],
  theta=['processing cost','mechanical properties','chemical stability', 'thermal stability',
           'device integration'],
  fill='toself'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=False
)

fig.show()

In [69]:
fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'polar'}]*2]*2)

fig.add_trace(go.Scatterpolar(
      name = "angular categories",
      r = [5, 4, 2, 4, 5],
      theta = ["a", "b", "c", "d", "a"],
    ), 1, 1)
fig.add_trace(go.Scatterpolar(
      name = "radial categories",
      r = ["a", "b", "c", "d", "b", "f", "a"],
      theta = [1, 4, 2, 1.5, 1.5, 6, 5],
      thetaunit = "radians",
    ), 1, 2)
fig.add_trace(go.Scatterpolar(
      name = "angular categories (w/ categoryarray)",
      r = [5, 4, 2, 4, 5],
      theta = ["a", "b", "c", "d", "a"],
    ), 2, 1)
fig.add_trace(go.Scatterpolar(
      name = "radial categories (w/ category descending)",
      r = ["a", "b", "c", "d", "b", "f", "a", "a"],
      theta = [45, 90, 180, 200, 300, 15, 20, 45],
    ), 2, 2)

fig.update_traces(fill='toself')
fig.update_layout(
    polar = dict(
      radialaxis_angle = -45,
      angularaxis = dict(
        direction = "clockwise",
        period = 6)
    ),
    polar2 = dict(
      radialaxis = dict(
        angle = 180,
        tickangle = -180 # so that tick labels are not upside down
      )
    ),
    polar3 = dict(
      sector = [80, 400],
      radialaxis_angle = -45,
      angularaxis_categoryarray = ["d", "a", "c", "b"]
    ),
    polar4 = dict(
      radialaxis_categoryorder = "category descending",
      angularaxis = dict(
        thetaunit = "radians",
        dtick = 0.3141592653589793
      ))
)

fig.show()

In [59]:
from google.colab import output
output.enable_custom_widget_manager()

In [60]:
import plotly.graph_objs as go
# import plotly.offline as py

import pandas as pd
import numpy as np
from ipywidgets import interactive, HBox, VBox

# py.init_notebook_mode()

df = pd.read_csv('https://raw.githubusercontent.com/jonmmease/plotly_ipywidget_notebooks/master/notebooks/data/cars/cars.csv')

f = go.FigureWidget([go.Scatter(y = df['City mpg'], x = df['City mpg'], mode = 'markers')])
scatter = f.data[0]
N = len(df)
scatter.x = scatter.x + np.random.rand(N)/10 *(df['City mpg'].max() - df['City mpg'].min())
scatter.y = scatter.y + np.random.rand(N)/10 *(df['City mpg'].max() - df['City mpg'].min())
scatter.marker.opacity = 0.5

def update_axes(xaxis, yaxis):
    scatter = f.data[0]
    scatter.x = df[xaxis]
    scatter.y = df[yaxis]
    with f.batch_update():
        f.layout.xaxis.title = xaxis
        f.layout.yaxis.title = yaxis
        scatter.x = scatter.x + np.random.rand(N)/10 *(df[xaxis].max() - df[xaxis].min())
        scatter.y = scatter.y + np.random.rand(N)/10 *(df[yaxis].max() - df[yaxis].min())

axis_dropdowns = interactive(update_axes, yaxis = df.select_dtypes('int64').columns, xaxis = df.select_dtypes('int64').columns)

# Create a table FigureWidget that updates on selection from points in the scatter plot of f
t = go.FigureWidget([go.Table(
    header=dict(values=['ID','Classification','Driveline','Hybrid'],
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[df[col] for col in ['ID','Classification','Driveline','Hybrid']],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])

def selection_fn(trace,points,selector):
    t.data[0].cells.values = [df.loc[points.point_inds][col] for col in ['ID','Classification','Driveline','Hybrid']]

scatter.on_selection(selection_fn)

# Put everything together
VBox((HBox(axis_dropdowns.children),f,t))

In [61]:
from IPython.display import display

In [62]:
display(VBox((HBox(axis_dropdowns.children),f,t)))

In [172]:
df.head()

,City mpg,Classification,Driveline,Engine Type,Fuel Type,Height,Highway mpg,Horsepower,Hybrid,ID,Length,Make,Model Year,Number of Forward Gears,Torque,Transmission,Width,Year
0,18,Automatic transmission,All-wheel drive,Audi 3.2L 6 cylinder 250hp 236ft-lbs,Gasoline,140,25,250,False,2009 Audi A3 3.2,143,Audi,2009 Audi A3,6,236,6 Speed Automatic Select Shift,202,2009
1,22,Automatic transmission,Front-wheel drive,Audi 2.0L 4 cylinder 200 hp 207 ft-lbs Turbo,Gasoline,140,28,200,False,2009 Audi A3 2.0 T AT,143,Audi,2009 Audi A3,6,207,6 Speed Automatic Select Shift,202,2009
2,21,Manual transmission,Front-wheel drive,Audi 2.0L 4 cylinder 200 hp 207 ft-lbs Turbo,Gasoline,140,30,200,False,2009 Audi A3 2.0 T,143,Audi,2009 Audi A3,6,207,6 Speed Manual,202,2009
3,21,Automatic transmission,All-wheel drive,Audi 2.0L 4 cylinder 200 hp 207 ft-lbs Turbo,Gasoline,140,28,200,False,2009 Audi A3 2.0 T Quattro,143,Audi,2009 Audi A3,6,207,6 Speed Automatic Select Shift,202,2009
4,21,Automatic transmission,All-wheel drive,Audi 2.0L 4 cylinder 200 hp 207 ft-lbs Turbo,Gasoline,140,28,200,False,2009 Audi A3 2.0 T Quattro,143,Audi,2009 Audi A3,6,207,6 Speed Automatic Select Shift,202,2009


In [173]:
f = go.FigureWidget([go.Scatter(y = df['City mpg'], x = df['City mpg'], mode = 'markers')])
scatter = f.data[0]
N = len(df)
scatter.x = scatter.x + np.random.rand(N)/10 *(df['City mpg'].max() - df['City mpg'].min())
scatter.y = scatter.y + np.random.rand(N)/10 *(df['City mpg'].max() - df['City mpg'].min())
scatter.marker.opacity = 0.5

In [64]:
def update_axes(xaxis, yaxis):
    scatter = f.data[0]
    scatter.x = df[xaxis]
    scatter.y = df[yaxis]
    with f.batch_update():
        f.layout.xaxis.title = xaxis
        f.layout.yaxis.title = yaxis
        scatter.x = scatter.x + np.random.rand(N)/10 *(df[xaxis].max() - df[xaxis].min())
        scatter.y = scatter.y + np.random.rand(N)/10 *(df[yaxis].max() - df[yaxis].min())

axis_dropdowns = interactive(update_axes, yaxis = df.select_dtypes('int64').columns, xaxis = df.select_dtypes('int64').columns)

In [66]:
# Create a table FigureWidget that updates on selection from points in the scatter plot of f
t = go.FigureWidget([go.Table(
    header=dict(values=['ID','Classification','Driveline','Hybrid'],
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[df[col] for col in ['ID','Classification','Driveline','Hybrid']],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])

# def selection_fn(trace,points,selector):
#     t.data[0].cells.values = [df.loc[points.point_inds][col] for col in ['ID','Classification','Driveline','Hybrid']]

# scatter.on_selection(selection_fn)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.7/dist-packages/pandas/core/indexes/base.py:3361 in get_loc               │
│                                                                                                  │
│   3358 │   │   │   │   )                                                                         │
│   3359 │   │   │   casted_key = self._maybe_cast_indexer(key)                                    │
│   3360 │   │   │   try:                                                                          │
│ ❱ 3361 │   │   │   │   return self._engine.get_loc(casted_key)                                   │
│   3362 │   │   │   except KeyError as err:                                                       │
│   3363 │   │   │   │   raise KeyError(key) from err                                              │
│   3364                                                                                           │
│                                                                                                  │
│ /content/funcs/pandas/_libs/index.pyx:76 in pandas._libs.index.IndexEngine.get_loc               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/content/funcs/pandas/_libs/index.pyx'                     │
│                                                                                                  │
│ /content/funcs/pandas/_libs/index.pyx:108 in pandas._libs.index.IndexEngine.get_loc              │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/content/funcs/pandas/_libs/index.pyx'                     │
│                                                                                                  │
│ /content/funcs/pandas/_libs/hashtable_class_helper.pxi:5198 in                                   │
│ pandas._libs.hashtable.PyObjectHashTable.get_item                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/content/funcs/pandas/_libs/hashtable_class_helper.pxi'    │
│                                                                                                  │
│ /content/funcs/pandas/_libs/hashtable_class_helper.pxi:5206 in                                   │
│ pandas._libs.hashtable.PyObjectHashTable.get_item                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/content/funcs/pandas/_libs/hashtable_class_helper.pxi'    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'ID'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-66-bdbf9b9b9247>:6 in <module>                                                    │
│ <ipython-input-66-bdbf9b9b9247>:6 in <listcomp>                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3458 in __getitem__                  │
│                                                                                                  │
│    3455 │   │   if is_single_key:                                                                │
│    3456 │   │   │   if self.columns.nlevels > 1:                                                 │
│    3457 │   │   │   │   return self._getitem_multilevel(key)                                     │
│ ❱  3458 │   │   │   indexer = self.columns.get_loc(key)                   